In [0]:
#!pip install Pandas pillow tensorflow pyspark pyarrow

In [0]:
import pandas as pd
from PIL import Image
import numpy as np
import io

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

2024-04-09 13:04:32.642476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
%fs ls /FileStore/tables/fruits/apples

path,name,size,modificationTime
dbfs:/FileStore/tables/fruits/apples/3_100.jpg,3_100.jpg,4815,1712262268000
dbfs:/FileStore/tables/fruits/apples/60_100.jpg,60_100.jpg,4623,1712262268000
dbfs:/FileStore/tables/fruits/apples/r_6_100.jpg,r_6_100.jpg,5450,1712262268000


In [0]:
PATH_DATA = "/FileStore/tables/fruits"
PATH_RESULT = "/FileStore/tables/fruits/results"

In [0]:
spark

Implémenter un workflow avec les étapes suivantes :
- importer les images dans un dataframe **pandas UDF**
- associer aux images leur **label**
- **redimensionner les images pour qu'elles soient compatibles avec notre modèle** (224, 224, 3)
- importer le modèle **MobileNetV2**
- créer un **nouveau modèle** sans la dernière couche de MobileNetV2
- définir le processus de chargement des images et l'application de leur featurisation à travers l'utilisation de pandas UDF. Pour cela utiliser la ressource suivante : https://github.com/tntn123/spark_transferlearning/blob/main/main.py
- exécuter les actions d'extraction de features
- enregistrer le résultat
(- éventuellement, tester le bon fonctionnement en chargeant les données enregistrées)

Charger les données

In [0]:
images = spark.read.format(
    "binaryFile"
).option(
    "pathGlobFilter", "*.jpg"
).option(
    "recursiveFileLookup", "true"
).load(PATH_DATA)


In [0]:
images = images.withColumn("label", element_at(split(images["path"], "/"), -2))

In [0]:
images.show()

+--------------------+-------------------+------+--------------------+--------+
|                path|   modificationTime|length|             content|   label|
+--------------------+-------------------+------+--------------------+--------+
|dbfs:/FileStore/t...|2024-04-04 20:24:28|  5450|[FF D8 FF E0 00 1...|  apples|
|dbfs:/FileStore/t...|2024-04-04 20:24:28|  4815|[FF D8 FF E0 00 1...|  apples|
|dbfs:/FileStore/t...|2024-04-04 20:24:28|  4623|[FF D8 FF E0 00 1...|  apples|
|dbfs:/FileStore/t...|2024-04-05 12:44:23|  4452|[FF D8 FF E0 00 1...|apricots|
|dbfs:/FileStore/t...|2024-04-05 12:44:23|  4450|[FF D8 FF E0 00 1...|apricots|
|dbfs:/FileStore/t...|2024-04-05 12:44:22|  4436|[FF D8 FF E0 00 1...|apricots|
|dbfs:/FileStore/t...|2024-04-05 12:44:22|  4147|[FF D8 FF E0 00 1...|apricots|
|dbfs:/FileStore/t...|2024-04-04 20:22:55|  3143|[FF D8 FF E0 00 1...| bananas|
|dbfs:/FileStore/t...|2024-04-04 20:22:55|  3143|[FF D8 FF E0 00 1...| bananas|
|dbfs:/FileStore/t...|2024-04-04 20:22:5

Charger le modèle MobileNetV2

In [0]:
model = MobileNetV2(input_shape=(224, 224, 3))

14536120/14536120 [==============================] - 1s 0us/step


Récupérer le backbone

In [0]:
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

Afficher l'architecture du modèle (summary)

In [0]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

Mettre ces traitements dans une fonction (qui sera distribuée). Réaliser la diffusion des poids du modèles (ici nécessaire)

In [0]:
bc_weights = sc.broadcast(new_model.get_weights())

In [0]:
def model_fn():
    model = MobileNetV2(input_shape=(224, 224, 3))
    backbone = Model(inputs=model.input, outputs=model.layers[-2].output)
    backbone.set_weights(bc_weights.value)
    return backbone

Récupérer les fonctions de preprocessing pour MobileNet au lien fourni : https://github.com/tntn123/spark_transferlearning/blob/main/main.py

In [0]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)


@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

/databricks/spark/python/pyspark/sql/pandas/functions.py:440: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


Si un jour vous rencontrez des erreurs de type Out Of Memory (OOM), réduisez la taille des lots.

In [0]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

Créer le dataframe des features. Redistribuer les données du dataframe en 2 partitions.

In [0]:
features_df = images.repartition(2).select(
    col("path"),
    col("label"),
    featurize_udf("content").alias("features")
)

Enregistrer les données traitées au format "**parquet**".

In [0]:
features_df.write.mode("overwrite").parquet(PATH_RESULT)

Charger les données fraichement enregistrées dans un dataframe

In [0]:
df = spark.read.parquet(PATH_RESULT)

Afficher les 5 premières lignes du dataframe

In [0]:
df.show()

+--------------------+--------+--------------------+
|                path|   label|            features|
+--------------------+--------+--------------------+
|dbfs:/FileStore/t...| bananas|[1.1165012, 0.173...|
|dbfs:/FileStore/t...| bananas|[0.49637008, 0.01...|
|dbfs:/FileStore/t...|  apples|[0.6281851, 0.039...|
|dbfs:/FileStore/t...| bananas|[1.1631975, 0.079...|
|dbfs:/FileStore/t...|apricots|[0.3087933, 0.253...|
|dbfs:/FileStore/t...| bananas|[1.5146621, 0.163...|
|dbfs:/FileStore/t...| bananas|[0.34093124, 2.20...|
|dbfs:/FileStore/t...| bananas|[0.26353976, 0.0,...|
|dbfs:/FileStore/t...|apricots|[0.7902153, 0.0, ...|
|dbfs:/FileStore/t...| bananas|[0.09224932, 0.0,...|
|dbfs:/FileStore/t...|apricots|[0.32175714, 0.14...|
|dbfs:/FileStore/t...| bananas|[0.26656345, 0.01...|
|dbfs:/FileStore/t...| bananas|[0.3504598, 0.0, ...|
|dbfs:/FileStore/t...| bananas|[1.2393048, 0.0, ...|
|dbfs:/FileStore/t...| bananas|[1.2731438, 0.075...|
|dbfs:/FileStore/t...| bananas|[1.2296529, 0.0

Vérifier la dimension du vecteur de caractéristiques des images (dimension attendue : 1280)

In [0]:
len(df.select("features").head()[0])

1280